In [3]:
import pandas as pd
import sqlite3

In [4]:
# Create Database Connection
conn = sqlite3.connect('../HomelessData.db')
c = conn.cursor()

In [5]:
# Create ShelterAvalibility dataframe and database
for sheet in ["2020","2019","2018","2017","2016","2015","2014","2013"]: 
    try: 
        hic_df = pd.read_excel("../resources/2007-2020-HIC-Counts-by-CoC.xlsx",sheet,skiprows=[0],skipfooter=1)

        # Adjust Column
        hic_df.rename(columns = {'CoC Number' : 'Coc','Total Year-Round Beds (ES, TH, SH)' : 'Total','Total Year-Round Beds (ES)' : 'Emergency','Total Year-Round Beds (TH)' : 'Transitional','Total Year-Round Beds (SH)' : 'SafeHaven','Total Year-Round Beds (RRH)' : 'RapidRehousing','Total Year-Round Beds (PSH)' : 'PermanentSupportive','Total Year-Round Beds (OPH)' : 'PermanentOther'},inplace=True)
        hic_df = hic_df[["Coc", "Total", "Emergency", "Transitional", "SafeHaven", "RapidRehousing", "PermanentSupportive", "PermanentOther"]]
        hic_df["Year"] = sheet

        # Insert Into DB
        hic_df.to_sql("shelterAvalibility",con=conn,if_exists = 'append',index=False)
        print(f"Gathered Data from {sheet}")
    except Exception as E:
        print(f"Couldnt get data from {sheet}")
print("Finished")

"['Transitional', 'PermanentOther', 'Total', 'RapidRehousing', 'PermanentSupportive', 'SafeHaven', 'Emergency'] not in index"
Couldnt get data from 2013


C:\Users\Jonat\anaconda3\lib\site-packages\openpyxl\worksheet\header_footer.py:48: UserWarning: Cannot parse header or footer so it will be ignored
  warn("""Cannot parse header or footer so it will be ignored""")


In [7]:
# Create PIT and CityData dataframe and database
for sheet in ["2020","2019","2018","2017","2016","2015","2014","2013"]: 
    try:
        pit_df = pd.read_excel("../resources/2007-2020-PIT-Estimates-by-CoC.xlsx",sheet,skipfooter=3)

        # Rename pit_df columns
        pit_df.rename(columns = {'CoC Number' : 'Coc',f'Overall Homeless, {sheet}' : "Total",f'Unsheltered Homeless, {sheet}' : "Unsheltered"},inplace=True)

        #Create CityData Database
        pit_df[["Coc","CoC Name"]].to_sql("CityData",con=conn,if_exists = "replace")

        # Create New Columns
        pit_df["Sheltered"] = pit_df["Total"]-pit_df["Unsheltered"]
        pit_df = pit_df[['Coc','Total','Unsheltered','Sheltered']]
        pit_df["Year"] = sheet

        # Create HomelessData table
        pit_df.to_sql("HomelessCounts",con=conn,if_exists='append',index=False)
        print(f"Gathered Data from {sheet}")
    except Exception as E : 
        print(f"Couldnt get data from {sheet}")
print("Finished")

Gathered Data from 2020
Gathered Data from 2019
Gathered Data from 2018
Gathered Data from 2017
Gathered Data from 2016
Gathered Data from 2015
Gathered Data from 2014
Gathered Data from 2013


In [9]:
# Create The Funding Database and Dataframe
for sheet in ["2020","2019","2018","2017","2016","2015","2014","2013"]:
    try:
        awards_df = pd.read_excel("../resources/2019-Awards-by-CoC-Component-Project.xlsx",sheet)
        awards_df.rename(columns = {"CoC Number" : "CoC"},inplace=True)
        awards_df = awards_df[["CoC","Component","Awarded Amount"]]
        awards_df["Year"] = sheet
        awards_df.to_sql("FundingData",con=conn,if_exists='append')
    except Exception as E : 
        print(f"Couldnt gather data from {sheet}")
        
print("Finished")

Couldnt gather data from 2020
Couldnt gather data from 2017
Couldnt gather data from 2016
Couldnt gather data from 2015
Couldnt gather data from 2014
Couldnt gather data from 2013
Finished


In [38]:
# Load in county data 
df = pd.read_csv("../resources/county_level_data/count_fips.csv")
coc_df = pd.read_csv("../resources/county_level_data/fips_Coc.csv")[["FIPS","number"]]
coc_df.rename(columns = {"number" : "Coc"},inplace=True)
df.merge(coc_df,on="FIPS")
df.to_sql("CityMeta",con=conn)

In [49]:
# Create the Coc Meta
df = pd.read_excel("../resources/2007-2020-PIT-Estimates-by-CoC.xlsx",skipfooter=3)[['CoC Number',"CoC Category"]]
df.to_sql("CocMeta",con=conn)

C:\Users\Jonat\anaconda3\lib\site-packages\pandas\core\generic.py:2779: UserWarning: The spaces in these column names will not be changed. In pandas versions < 0.14, spaces were converted to underscores.
  sql.to_sql(


,CoC Number,CoC Category
0,AK-500,Other Largely Urban CoC
1,AK-501,Largely Rural CoC
2,AL-500,Largely Suburban CoC
3,AL-501,Other Largely Urban CoC
4,AL-502,Largely Rural CoC
...,...,...
387,WV-500,Largely Rural CoC
388,WV-501,Largely Rural CoC
389,WV-503,Largely Suburban CoC
390,WV-508,Largely Rural CoC
